#**Model Attempt #3**

Improvments To Be Made:

Image size- 224×224 (ResNet)

Min data aug- flip/rotate

Fix Class imbalance w/ weighted loss

Early stopping

In [ ]:
# Load in libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
import random
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import os
from imageio import imread

# Stop warnings
import warnings
warnings.filterwarnings('ignore')

#from google.colab import drive
#drive.mount('/content/drive')
#!unzip /content/drive/MyDrive/dataset.zip -d /content/dataset
#path = '/content/dataset/dataset'


In [ ]:
# Load data
width = []
height = []
channel_color = []
weather_type = []

path = '/content/dataset/dataset'

# for each subfolder (category)
for folder in os.listdir(path):
    subfolder_path = os.path.join(path, folder_name)

    # skip if not a directory
    if not os.path.isdir(subfolder_path):
        continue

    for image_file in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_file)

        # skip directories inside category folders
        if os.path.isdir(image_path):
            continue

        image = imread(image_path)

        # handle grayscale images
        if len(image.shape) < 3:
            image = image.reshape(image.shape + (1,))

        height, width, channel = image.shape
        width.append(width)      # width on x-axis
        height.append(height)     # height on y-axis
        channel_color.append(channel)
        weather_type.append(folder_name)  # use folder name as label

# create DataFrame
df = pd.DataFrame({
    'width': width,
    'height': height,
    'channels': channel_color,
    'weather': weather_type
})

df.head()

Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from collections import Counter
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data augmentation and transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # pretrained ImageNet mean/std
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset
dataset = datasets.ImageFolder('/content/dataset/dataset', transform=transform_train)
num_classes = len(dataset.classes)
print("Classes:", dataset.classes)

# Optional: check class distribution
labels = [label for _, label in dataset]
counts = Counter(labels)
print("Images per class:", counts)

# Train/Validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Override val dataset transforms
val_dataset.dataset.transform = transform_val

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

#Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier (fc) layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Weighted loss for class imbalance
class_counts = torch.tensor([counts[i] for i in range(num_classes)], dtype=torch.float)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * num_classes  # normalize
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# Optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# Training loop with early stopping
epochs = 15
best_val_acc = 0
best_model_wts = copy.deepcopy(model.state_dict())
patience_counter = 0
early_stop_patience = 4

for epoch in range(epochs):
    # Training
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("Early stopping triggered")
            break

# Load best weights
model.load_state_dict(best_model_wts)
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")